<a href="https://colab.research.google.com/github/rafaellccouto/postech_analise_vinhos_comercio/blob/main/Analise_vinhos_comercio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analise de dados comerciais de vinhos para planos estratégicos de investimentos

###  Alguns esclarecimentos se fazem necessários, para que os usuários façam o uso correto das informações:

    Os vinhos nacionais são classificados para fins estatísticos em vinho de mesa (elaborados com uvas americanas e/ou híbridas), vinho fino de mesa (elaborados com uvas Vitis Vinifera L.) e vinho especial (corte de vinho de mesa e fino de mesa).
    Os vinhos importados, denominados de vinhos de mesa são equivalentes aos vinhos finos de mesa nacionais, pois são elaborados com uvas Vitis Vinifera L.
    Os dados constantes da base de dados ALICEweb, referentes à vinhos e espumantes são expressos em quilos, no entanto considerando que a densidade desses produtos é de aproximadamente um (1), consideramos 1 Kg = 1L.
    Os arquivos de download possuem a extensão CSV, para facilitar a importação em planilhas ou banco de dados.


In [1]:
# Análise de Dados da Cadeia Produtiva de Vinhos
# Bibliotecas básicas
import pandas as pd
import numpy as np
import os as os
from sklearn.linear_model import LinearRegression

# Bibliotecas de visualização
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Configurações iniciais
pd.set_option('future.no_silent_downcasting', True)
plt.style.use('ggplot')  # Estilo mais agradável para os gráficos
sns.set_palette("husl")  # Paleta de cores harmoniosa

# Carregamento dos dados com tratamento de erros robusto
def carregar_dados(caminho):
    try:
        df = pd.read_csv(caminho, sep=';', encoding='utf-8', 
                        on_bad_lines='warn', 
                        thousands='.', 
                        decimal=',')
        print(f"Arquivo {caminho} carregado com sucesso. Shape: {df.shape}")
        return df
    except Exception as e:
        print(f"Erro ao carregar {caminho}: {str(e)}")
        return None

# Carregando todos os dataframes
dfs = {
    'comercializacao': carregar_dados('Comercializacao.csv'),
    'exportacao': carregar_dados('Exportacao.csv'),
    'importacao': carregar_dados('Importacao.csv'),
    'processamento': carregar_dados('Processamento.csv'),
    'producao': carregar_dados('Producao.csv')
}

df_comercializacao = dfs['comercializacao']
df_exportacao = dfs['exportacao']
df_importacao = dfs['importacao']
df_processamento = dfs['processamento']
df_producao = dfs['producao']



# Função para análise exploratória inicial rápida
def analise_rapida(df, nome):
    if df is not None:
        print(f"\n{'='*50}")
        print(f"Análise Exploratória: {nome}")
        print(f"{'='*50}")
        
        # Info básica
        print("\nInformações do DataFrame:")
        display(df.info())
        
        # Primeiras linhas
        print("\nPrimeiras linhas:")
        display(df.head())
        
        # Estatísticas descritivas
        print("\nEstatísticas descritivas:")
        display(df.describe(include='all'))
        
        # Valores ausentes
        print("\nValores ausentes por coluna:")
        display(df.isna().sum())
        
# Executando análise para cada dataframe
for nome, df in dfs.items():
    analise_rapida(df, nome)


Arquivo Comercializacao.csv carregado com sucesso. Shape: (62, 57)
Arquivo Exportacao.csv carregado com sucesso. Shape: (137, 110)
Arquivo Importacao.csv carregado com sucesso. Shape: (68, 110)
Arquivo Processamento.csv carregado com sucesso. Shape: (139, 57)
Arquivo Producao.csv carregado com sucesso. Shape: (51, 57)

Análise Exploratória: comercializacao

Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 57 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       62 non-null     int64 
 1   control  61 non-null     object
 2   Produto  62 non-null     object
 3   1970     62 non-null     int64 
 4   1971     62 non-null     int64 
 5   1972     62 non-null     int64 
 6   1973     62 non-null     int64 
 7   1974     62 non-null     int64 
 8   1975     62 non-null     int64 
 9   1976     62 non-null     int64 
 10  1977     62 non-null     int64 
 11  1978     62 non-null     

None


Primeiras linhas:


,id,control,Produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,98327606,114399031,118377367,116617910,94173324,108031792,139238614,...,206404427,209198468,166769622,176059959,177186273,180446489,215557931,210012238,187939996,187016848
1,2,vm_Tinto,Tinto,83300735,98522869,101167932,98196747,77167303,91528090,116407222,...,178250072,182028785,146646365,154309442,155115499,158519218,189573423,185653678,165067340,165097539
2,3,vm_Rosado,Rosado,107681,542274,7770851,8425617,8891367,7261777,11748047,...,1419855,1409002,1391942,1097426,1972944,1265435,1394901,1931606,2213723,2520748
3,4,vm_Branco,Branco,14919190,15333888,9438584,9995546,8114654,9241925,11083345,...,26734500,25760681,18731315,20653091,20097830,20661836,24589607,22426954,20658933,19398561
4,5,VINHO FINO DE MESA,VINHO FINO DE MESA,4430629,4840369,5602091,7202830,7571802,8848303,14095648,...,20424983,20141631,19630158,15874354,14826143,15684588,24310834,27080445,21533487,18589310



Estatísticas descritivas:


...

,id,control,produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,217208604,154264651,146953297,116710345,193875345,177401209,144565438,...,196173123,210308560,86319015,255015187,218375636,144629737,124200414,173899995,195031611,169762429
1,2,vm_Tinto,Tinto,174224052,121133369,118180926,88589019,146544484,144274134,118360170,...,157776363,169811472,75279191,1365957,188270142,121045115,103916391,146075996,162844214,139320884
2,3,vm_Branco,Branco,748400,1160500,1812367,243900,4138768,1441507,1871473,...,37438069,39557250,10727099,217527985,29229970,22032828,19568734,26432799,30198430,27910299
3,4,vm_Rosado,Rosado,42236152,31970782,26960004,27877426,43192093,31685568,24333795,...,958691,939838,312725,36121245,875524,1551794,715289,1391200,1988968,2531246
4,5,VINHO FINO DE MESA (VINIFERA),VINHO FINO DE MESA (VINIFERA),23899346,23586062,21078771,12368410,31644124,39424590,34500590,...,38464314,37148982,18070626,44537870,38707220,37615422,32516686,43474998,47511796,46268556



Estatísticas descritivas:


,id,control,produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
count,51.000000,51,51,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,...,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01,5.100000e+01
unique,NaN,51,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,VINHO DE MESA,Tinto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,26.000000,NaN,NaN,1.005373e+07,7.585407e+06,7.210040e+06,6.172419e+06,1.072253e+07,1.060623e+07,7.921217e+06,...,1.467719e+07,1.736085e+07,7.864719e+06,1.905223e+07,1.638247e+07,1.580154e+07,1.249067e+07,1.911132e+07,1.752368e+07,1.795266e+07
std,14.866069,NaN,NaN,3.880896e+07,2.744967e+07,2.628082e+07,2.089229e+07,3.480845e+07,3.317688e+07,2.642639e+07,...,3.798277e+07,4.353093e+07,1.940315e+07,5.084260e+07,4.379813e+07,3.833873e+07,2.914619e+07,4.571075e+07,4.352044e+07,4.426636e+07
min,1.000000,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,13.500000,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.210000e+02,0.000000e+00,0.000000e+00
50%,26.000000,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.700000e+03,9.220000e+02,0.000000e+00,3.000000e+03,2.000000e+03,3.251900e+04,4.600000e+04,3.000000e+04,5.000000e+03,1.720000e+04
75%,38.500000,NaN,NaN,8.956600e+05,8.658000e+05,1.182465e+06,3.784000e+05,2.798100e+05,1.235000e+05,2.467500e+05,...,1.504279e+06,1.536993e+06,1.033048e+06,1.759016e+06,1.533595e+06,1.757118e+06,2.747322e+06,3.824400e+06,3.138016e+06,3.974682e+06



Valores ausentes por coluna:


id         0
control    0
produto    0
1970       0
1971       0
1972       0
1973       0
1974       0
1975       0
1976       0
1977       0
1978       0
1979       0
1980       0
1981       0
1982       0
1983       0
1984       0
1985       0
1986       0
1987       0
1988       0
1989       0
1990       0
1991       0
1992       0
1993       0
1994       0
1995       0
1996       0
1997       0
1998       0
1999       0
2000       0
2001       0
2002       0
2003       0
2004       0
2005       0
2006       0
2007       0
2008       0
2009       0
2010       0
2011       0
2012       0
2013       0
2014       0
2015       0
2016       0
2017       0
2018       0
2019       0
2020       0
2021       0
2022       0
2023       0
dtype: int64

## Filtragem básica de dados: Últimos 15 anos (2009 - 2023), zerar dados nulos e consolidar dados semestrais em um ano só.

### Os valores nulos podem ser 0 pois representa a sua insignificância por não ter registro da EMBRAPA e não é possível valores menores que 0 para as grandezas analisadas

### Comercialização

In [2]:
comercializacaofiltrada = df_comercializacao.copy()

# Identificar colunas de ano a serem removidas (1970 a 2008)
anos_para_remover = [str(ano) for ano in range(1970, 2009)]

# Filtrar apenas as colunas de ano que existem no DataFrame
colunas_para_remover = [coluna for coluna in anos_para_remover if coluna in comercializacaofiltrada.columns]

# Remover as colunas
comercializacaofiltrada = comercializacaofiltrada.drop(columns=colunas_para_remover, errors='ignore')

# Substituir valores nulos e NaN por 0
comercializacaofiltrada = comercializacaofiltrada.fillna(0)

print("Primeiras linhas do DataFrame após remoção dos anos anteriores a 2009 e substituição de nulos/NaN por 0:")
print(comercializacaofiltrada.head())


Primeiras linhas do DataFrame após remoção dos anos anteriores a 2009 e substituição de nulos/NaN por 0:
   id              control              Produto       2009       2010  \
0   1        VINHO DE MESA        VINHO DE MESA  234525979  221242945   
1   2             vm_Tinto                Tinto  193004182  188649074   
2   3            vm_Rosado               Rosado    2307580    2036928   
3   4            vm_Branco               Branco   39214217   30556943   
4   5  VINHO  FINO DE MESA  VINHO  FINO DE MESA   33080270   21390159   

        2011       2012       2013       2014       2015       2016  \
0  230310468  206969571  221590810  206404427  209198468  166769622   
1  196562722  173964776  188033494  178250072  182028785  146646365   
2    1668823    1738134    1777648    1419855    1409002    1391942   
3   32078923   31266661   31779668   26734500   25760681   18731315   
4   19967310   22469950   27912934   20424983   20141631   19630158   

        2017       2018      

In [3]:
comercializacaofiltrada.head()

,id,control,Produto,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,234525979,221242945,230310468,206969571,221590810,206404427,209198468,166769622,176059959,177186273,180446489,215557931,210012238,187939996,187016848
1,2,vm_Tinto,Tinto,193004182,188649074,196562722,173964776,188033494,178250072,182028785,146646365,154309442,155115499,158519218,189573423,185653678,165067340,165097539
2,3,vm_Rosado,Rosado,2307580,2036928,1668823,1738134,1777648,1419855,1409002,1391942,1097426,1972944,1265435,1394901,1931606,2213723,2520748
3,4,vm_Branco,Branco,39214217,30556943,32078923,31266661,31779668,26734500,25760681,18731315,20653091,20097830,20661836,24589607,22426954,20658933,19398561
4,5,VINHO FINO DE MESA,VINHO FINO DE MESA,33080270,21390159,19967310,22469950,27912934,20424983,20141631,19630158,15874354,14826143,15684588,24310834,27080445,21533487,18589310


In [4]:
arquivo = 'comercializacao_filtrada.csv'
if not os.path.exists(arquivo):
    comercializacaofiltrada.to_csv(arquivo, index=False)
    print(f"DataFrame exportado para '{arquivo}'.")
else:
    print(f"Arquivo '{arquivo}' já está gerado.")


Arquivo 'comercializacao_filtrada.csv' já está gerado.


### Exportação

In [5]:
def split_dataframe(df):
    """
    Separa o DataFrame original em dois:
    1. exportacao_qtd: contém apenas os anos de 2009-2023 (sem sufixo .1)
    2. exportacao_valor: contém os anos de 2009-2023 com sufixo .1 (removendo o '.1' dos nomes)
    
    Parâmetros:
    df -- DataFrame original
    
    Retorna:
    Tuple com (exportacao_qtd, exportacao_valor)
    """
    # Criar listas de colunas para cada DataFrame
    cols_qtd = [col for col in df.columns if not col.endswith('.1') and col.isdigit()]
    cols_valor = [col for col in df.columns if col.endswith('.1')]
    
    # Filtrar apenas colunas de 2009-2023
    years_range = set(str(year) for year in range(2009, 2024))
    
    cols_qtd = [col for col in cols_qtd if col in years_range]
    cols_valor = [col for col in cols_valor if col[:-2] in years_range]
    
    # Criar DataFrames separados mantendo todas as outras colunas não-numéricas
    other_cols = [col for col in df.columns if not (col.isdigit() or col.endswith('.1'))]
    
    exportacao_qtd = df[other_cols + cols_qtd].copy()
    
    # Para exportacao_valor, remover o '.1' dos nomes das colunas
    exportacao_valor = df[other_cols + cols_valor].copy()
    exportacao_valor.columns = other_cols + [col[:-2] for col in cols_valor]
    
    return exportacao_qtd, exportacao_valor

# Separar os DataFrames
exportacao_qtd, exportacao_valor = split_dataframe(df_exportacao)

# Exibir informações sobre os novos DataFrames
print("\nDataFrame 'exportacao_qtd':")
print(exportacao_qtd.head())

print("\nDataFrame 'exportacao_valor':")
print(exportacao_valor.head())



DataFrame 'exportacao_qtd':
   Id                             País    2009   2010   2011  2012   2013  \
0   1                      Afeganistão       0      0      0     0      0   
1   2                    África do Sul       0      0      0     0      0   
2   3  Alemanha, República Democrática  225086  27715  36070  8189  61699   
3   4                           Angola   54786  33557  13889  2833   1573   
4   5                         Anguilla       0      0      0     0      0   

     2014   2015   2016   2017   2018  2019  2020  2021  2022  2023  
0       0      0      0      0      0     0     0    11     0     0  
1       0      0      0      0      0    26     4     0     0   117  
2  213348  10680  14012  15467  10794  3660  6261  2698  7630  4806  
3   12182   1908   7359  10170    477   345     0     0  4068     0  
4       0      0      0      0      0     0     0     0     0     0  

DataFrame 'exportacao_valor':
   Id                             País    2009    2010   

In [6]:
# Verifica e exporta o DataFrame de quantidades
arquivo_qtd = 'exportacao_qtd.csv'
if not os.path.exists(arquivo_qtd):
    exportacao_qtd.to_csv(arquivo_qtd, index=False)
    print(f"DataFrame 'exportacao_qtd' exportado para '{arquivo_qtd}'.")
else:
    print(f"Arquivo '{arquivo_qtd}' já existe.")

# Verifica e exporta o DataFrame de valores
arquivo_valor = 'exportacao_valor.csv'
if not os.path.exists(arquivo_valor):
    exportacao_valor.to_csv(arquivo_valor, index=False)
    print(f"DataFrame 'exportacao_valor' exportado para '{arquivo_valor}'.")
else:
    print(f"Arquivo '{arquivo_valor}' já existe.")


Arquivo 'exportacao_qtd.csv' já existe.
Arquivo 'exportacao_valor.csv' já existe.


In [7]:
# Ler as 5 primeiras linhas do arquivo de quantidades
print("\nPrimeiras 5 linhas de exportacao_qtd.csv:")
qtd_head = pd.read_csv('exportacao_qtd.csv', nrows=5)
print(qtd_head)

# Ler as 5 primeiras linhas do arquivo de valores
print("\nPrimeiras 5 linhas de exportacao_valor.csv:")
valor_head = pd.read_csv('exportacao_valor.csv', nrows=5)
print(valor_head)



Primeiras 5 linhas de exportacao_qtd.csv:
   Id                             País    2009   2010   2011  2012   2013  \
0   1                      Afeganistão       0      0      0     0      0   
1   2                    África do Sul       0      0      0     0      0   
2   3  Alemanha, República Democrática  225086  27715  36070  8189  61699   
3   4                           Angola   54786  33557  13889  2833   1573   
4   5                         Anguilla       0      0      0     0      0   

     2014   2015   2016   2017   2018  2019  2020  2021  2022  2023  
0       0      0      0      0      0     0     0    11     0     0  
1       0      0      0      0      0    26     4     0     0   117  
2  213348  10680  14012  15467  10794  3660  6261  2698  7630  4806  
3   12182   1908   7359  10170    477   345     0     0  4068     0  
4       0      0      0      0      0     0     0     0     0     0  

Primeiras 5 linhas de exportacao_valor.csv:
   Id                        

### Importação

In [8]:
def split_dataframe(df):
    """
    Separa o DataFrame original em dois:
    1. importacao_qtd: contém apenas os anos de 2009-2023 (sem sufixo .1)
    2. importacao_valor: contém os anos de 2009-2023 com sufixo .1 (removendo o '.1' dos nomes)
    
    Parâmetros:
    df -- DataFrame original
    
    Retorna:
    Tuple com (importacao_qtd, importacao_valor)
    """
    # Criar listas de colunas para cada DataFrame
    cols_qtd = [col for col in df.columns if not col.endswith('.1') and col.isdigit()]
    cols_valor = [col for col in df.columns if col.endswith('.1')]
    
    # Filtrar apenas colunas de 2009-2023
    years_range = set(str(year) for year in range(2009, 2024))
    
    cols_qtd = [col for col in cols_qtd if col in years_range]
    cols_valor = [col for col in cols_valor if col[:-2] in years_range]
    
    # Criar DataFrames separados mantendo todas as outras colunas não-numéricas
    other_cols = [col for col in df.columns if not (col.isdigit() or col.endswith('.1'))]
    
    importacao_qtd = df[other_cols + cols_qtd].copy()
    
    # Para importacao_valor, remover o '.1' dos nomes das colunas
    importacao_valor = df[other_cols + cols_valor].copy()
    importacao_valor.columns = other_cols + [col[:-2] for col in cols_valor]
    
    return importacao_qtd, importacao_valor

# Separar os DataFrames
importacao_qtd, importacao_valor = split_dataframe(df_importacao)

# Exibir informações sobre os novos DataFrames
print("\nDataFrame 'importacao_qtd':")
print(importacao_qtd.head())

print("\nDataFrame 'importacao_valor':")
print(importacao_valor.head())



DataFrame 'importacao_qtd':
   Id            País      2009      2010      2011      2012      2013  \
0   1   Africa do Sul    493093    966276    507509    722327    475331   
1   2        Alemanha     56075    192661    198025    113243    114866   
2   3         Argélia         0         0         0         0         0   
3   4  Arábia Saudita         0         0         0         0         0   
4   5       Argentina  13902750  16965266  16673491  14613839  12732815   

       2014      2015      2016      2017      2018      2019      2020  \
0    722715    743335    578829   1138732   1127053   1092042    627150   
1    115804     95171    158386     83289    142971    101055    136992   
2         0         0         0         0         0         0         0   
3         0         0         0         0       563         0         0   
4  13437321  12465041  14098009  15461740  15221318  16548931  22610267   

       2021      2022      2023  
0    859169    738116    522733  
1

In [9]:
# Verifica e exporta o DataFrame de quantidades
arquivo_qtd = 'importacao_qtd.csv'
if not os.path.exists(arquivo_qtd):
    importacao_qtd.to_csv(arquivo_qtd, index=False)
    print(f"DataFrame 'importacao_qtd' exportado para '{arquivo_qtd}'.")
else:
    print(f"Arquivo '{arquivo_qtd}' já existe.")

# Verifica e exporta o DataFrame de valores
arquivo_valor = 'importacao_valor.csv'
if not os.path.exists(arquivo_valor):
    importacao_valor.to_csv(arquivo_valor, index=False)
    print(f"DataFrame 'importacao_valor' exportado para '{arquivo_valor}'.")
else:
    print(f"Arquivo '{arquivo_valor}' já existe.")


Arquivo 'importacao_qtd.csv' já existe.
Arquivo 'importacao_valor.csv' já existe.


In [10]:
# Ler as 5 primeiras linhas do arquivo de quantidades
print("\nPrimeiras 5 linhas de importacao_qtd.csv:")
qtd_head = pd.read_csv('importacao_qtd.csv', nrows=5)
print(qtd_head)

# Ler as 5 primeiras linhas do arquivo de valores
print("\nPrimeiras 5 linhas de importacao_valor.csv:")
valor_head = pd.read_csv('importacao_valor.csv', nrows=5)
print(valor_head)



Primeiras 5 linhas de importacao_qtd.csv:
   Id            País      2009      2010      2011      2012      2013  \
0   1   Africa do Sul    493093    966276    507509    722327    475331   
1   2        Alemanha     56075    192661    198025    113243    114866   
2   3         Argélia         0         0         0         0         0   
3   4  Arábia Saudita         0         0         0         0         0   
4   5       Argentina  13902750  16965266  16673491  14613839  12732815   

       2014      2015      2016      2017      2018      2019      2020  \
0    722715    743335    578829   1138732   1127053   1092042    627150   
1    115804     95171    158386     83289    142971    101055    136992   
2         0         0         0         0         0         0         0   
3         0         0         0         0       563         0         0   
4  13437321  12465041  14098009  15461740  15221318  16548931  22610267   

       2021      2022      2023  
0    859169    738116

### Processamento

In [11]:
non_year_cols = ['id', 'control', 'cultivar']
year_cols = [str(year) for year in range(2009, 2024)]

# Combine non-year columns and desired year columns
columns_to_keep = non_year_cols + year_cols

# Filter the DataFrame to include only the specified columns
df_processamento_filtrado = df_processamento[columns_to_keep].copy()

# Process year columns
for col in year_cols:
    if col in df_processamento_filtrado.columns:
        # Convert to string to handle various data types, replace ',' with '.'
        df_processamento_filtrado[col] = df_processamento_filtrado[col].astype(str).str.replace(',', '.', regex=False)
        # Replace 'nd' and '*' with NaN. The FutureWarning is noted but the code is functional.
        df_processamento_filtrado[col] = df_processamento_filtrado[col].replace(['nd', '*'], np.nan)
        # Convert to numeric, coercing errors to NaN
        df_processamento_filtrado[col] = pd.to_numeric(df_processamento_filtrado[col], errors='coerce')
        # Fill NaN values with 0
        df_processamento_filtrado[col] = df_processamento_filtrado[col].fillna(0)

print("DataFrame 'df_processamento_filtrado' após filtragem e tratamento:")
print(df_processamento_filtrado.head())

DataFrame 'df_processamento_filtrado' após filtragem e tratamento:
   id               control           cultivar      2009      2010      2011  \
0   1                TINTAS             TINTAS  39148123  23633831  44473588   
1   2  ti_Alicante Bouschet  Alicante Bouschet   1652912    849263   2130579   
2   3           ti_Ancelota           Ancelota   1370728    853718   1346552   
3   4             ti_Aramon             Aramon         0         0         0   
4   5        ti_Alfrocheiro        Alfrocheiro     31168         0      4320   

       2012      2013      2014      2015      2016      2017      2018  2019  \
0  39303313  36855419  29810706  29935627  13370866  32850915  26868514   0.0   
1   2098824   1524728   1456305   1519576    908841   2040198   2103844   0.0   
2   1274677   1137943    937844    773526    179028    733907    492106   0.0   
3         0         0         0         0         0         0         0   0.0   
4         0         0         0         0      

In [12]:
arquivo = 'processamento_filtrado.csv'
if not os.path.exists(arquivo):
    df_processamento.to_csv(arquivo, index=False)
    print(f"DataFrame 'df_processamento' exportado para '{arquivo}'.")
else:
    print(f"Arquivo '{arquivo}' já está gerado.")

Arquivo 'processamento_filtrado.csv' já está gerado.


In [13]:
df_processamento_carregado = pd.read_csv('processamento_filtrado.csv')
print("\nDataFrame 'df_processamento_carregado' após carregamento do CSV:")
print(df_processamento_carregado.head())


DataFrame 'df_processamento_carregado' após carregamento do CSV:
   id               control           cultivar      2009      2010      2011  \
0   1                TINTAS             TINTAS  39148123  23633831  44473588   
1   2  ti_Alicante Bouschet  Alicante Bouschet   1652912    849263   2130579   
2   3           ti_Ancelota           Ancelota   1370728    853718   1346552   
3   4             ti_Aramon             Aramon         0         0         0   
4   5        ti_Alfrocheiro        Alfrocheiro     31168         0      4320   

       2012      2013      2014      2015      2016      2017      2018  2019  \
0  39303313  36855419  29810706  29935627  13370866  32850915  26868514   0.0   
1   2098824   1524728   1456305   1519576    908841   2040198   2103844   0.0   
2   1274677   1137943    937844    773526    179028    733907    492106   0.0   
3         0         0         0         0         0         0         0   0.0   
4         0         0         0         0       

### Produção

In [14]:
non_year_cols = ['id', 'control', 'produto']
year_cols = [str(year) for year in range(2009, 2024)]

# Combina colunas não-anuais e colunas anuais desejadas
columns_to_keep = non_year_cols + year_cols

# Filtra o DataFrame para incluir apenas as colunas especificadas
df_producao_filtrado = df_producao[columns_to_keep].copy()

# Processa colunas anuais
for col in year_cols:
    if col in df_producao_filtrado.columns:
        # Converte para string para lidar com vários tipos de dados, substitui ',' por '.'
        df_producao_filtrado[col] = df_producao_filtrado[col].astype(str).str.replace(',', '.', regex=False)
        # Substitui 'nd' e '*' por NaN
        df_producao_filtrado[col] = df_producao_filtrado[col].replace(['nd', '*'], np.nan)
        # Converte para numérico, forçando erros para NaN
        df_producao_filtrado[col] = pd.to_numeric(df_producao_filtrado[col], errors='coerce')
        # Preenche valores NaN com 0
        df_producao_filtrado[col] = df_producao_filtrado[col].fillna(0)

print("DataFrame 'df_producao_filtrado' após filtragem e tratamento:")
print(df_producao_filtrado.head())


DataFrame 'df_producao_filtrado' após filtragem e tratamento:
   id                        control                        produto  \
0   1                  VINHO DE MESA                  VINHO DE MESA   
1   2                       vm_Tinto                          Tinto   
2   3                      vm_Branco                         Branco   
3   4                      vm_Rosado                         Rosado   
4   5  VINHO FINO DE MESA (VINIFERA)  VINHO FINO DE MESA (VINIFERA)   

        2009       2010       2011       2012       2013       2014  \
0  205418206  195267980  257840749  212777037  196904222  196173123   
1  164143454  157290088  210113358  175875432  163111797  157776363   
2   39211278   35408083   46007504   34938249   32066403   37438069   
3    2063474    2569809    1719887    1963356    1726022     958691   
4   39900568   24805713   47598471   45200730   45782530   38464314   

        2015      2016       2017       2018       2019       2020       2021  \
0  

In [15]:
arquivo = 'producao_filtrada.csv'
if not os.path.exists(arquivo):
    df_producao.to_csv(arquivo, index=False)
    print(f"DataFrame 'df_producao' exportado para '{arquivo}'.")
else:
    print(f"Arquivo '{arquivo}' já está gerado.")

Arquivo 'producao_filtrada.csv' já está gerado.


In [16]:
df_producao_carregado = pd.read_csv('producao_filtrada.csv')
print("\nDataFrame 'df_producao_carregado' após carregamento do CSV:")
print(df_producao_carregado.head())


DataFrame 'df_producao_carregado' após carregamento do CSV:
   id                        control                        produto  \
0   1                  VINHO DE MESA                  VINHO DE MESA   
1   2                       vm_Tinto                          Tinto   
2   3                      vm_Branco                         Branco   
3   4                      vm_Rosado                         Rosado   
4   5  VINHO FINO DE MESA (VINIFERA)  VINHO FINO DE MESA (VINIFERA)   

        2009       2010       2011       2012       2013       2014  \
0  205418206  195267980  257840749  212777037  196904222  196173123   
1  164143454  157290088  210113358  175875432  163111797  157776363   
2   39211278   35408083   46007504   34938249   32066403   37438069   
3    2063474    2569809    1719887    1963356    1726022     958691   
4   39900568   24805713   47598471   45200730   45782530   38464314   

        2015      2016       2017       2018       2019       2020       2021  \
0  2

## Análise com métodos estatísticos




### Regressão Linear

### Exportação

In [17]:
# 1. Identar colunas de  2009 a 2023
year_cols = [str(year) for year in range(2009, 2024)]

# 2. Melt df_exportacao_filtrada para vetor
df_exportacao_long = exportacao_qtd.melt(
    id_vars=['Id', 'País'],
    value_vars=year_cols,
    var_name='Ano',
    value_name='Valor_Exportado'
)

# 3. Converter ano como inteiro
df_exportacao_long['Ano'] = pd.to_numeric(df_exportacao_long['Ano'])

print("DataFrame 'df_exportacao_long' after melting:")
print(df_exportacao_long.head())

# 4. Criar listas vazias para guardar os resultados
country_names = []
angular_coefficients = []
linear_coefficients = []
linear_equations = []

# 5. Iterar cada país
for country in df_exportacao_long['País'].unique():
    # 6a. Filtrar por país
    country_df = df_exportacao_long[df_exportacao_long['País'] == country]

    # 6b. Verificar se há dados suficiente para regressão
    if len(country_df) < 2:
        country_names.append(country)
        angular_coefficients.append(None)
        linear_coefficients.append(None)
        linear_equations.append(f"{country}: Insufficient data for regression")
        continue

    # 6c. Preparar os dados
    X = country_df['Ano'].values.reshape(-1, 1)  # Independent variable (Ano)
    y = country_df['Valor_Exportado'].values    # Dependent variable (Valor_Exportado)

    # 6d. Ajuste
    model = LinearRegression()
    model.fit(X, y)

    # 6e. Coeficientes
    coef_angular = model.coef_[0]
    coef_linear = model.intercept_

    # 6f. Formatar equação
    equation = f"y = {coef_angular:.2f}x + {coef_linear:.2f}"

    # 6g. Guardar nas listas
    country_names.append(country)
    angular_coefficients.append(coef_angular)
    linear_coefficients.append(coef_linear)
    linear_equations.append(equation)

# Criar um novo DataFrame com os resultados
df_regression_results = pd.DataFrame({
    'País': country_names,
    'Coeficiente_Angular': angular_coefficients,
    'Coeficiente_Linear': linear_coefficients,
    'Equacao_Linear': linear_equations
})

print("\nFirst 5 rows of the regression results DataFrame:")
print(df_regression_results.head())

# Exportar como CSV
arquivo = 'exportacao_reg_qtd.csv'
if not os.path.exists(arquivo):
    df_regression_results.to_csv(arquivo, index=False)
    print(f"DataFrame 'exportacao_reg_qtd' exportado para '{arquivo}'.")
else:
    print(f"Arquivo '{arquivo}' já está gerado.")

DataFrame 'df_exportacao_long' after melting:
   Id                             País   Ano  Valor_Exportado
0   1                      Afeganistão  2009                0
1   2                    África do Sul  2009                0
2   3  Alemanha, República Democrática  2009           225086
3   4                           Angola  2009            54786
4   5                         Anguilla  2009                0

First 5 rows of the regression results DataFrame:
                              País  Coeficiente_Angular  Coeficiente_Linear  \
0                      Afeganistão             0.196429       -3.952667e+02   
1                    África do Sul             3.260714       -6.563800e+03   
2  Alemanha, República Democrática         -8612.428571        1.740586e+07   
3                           Angola         -2357.303571        4.761867e+06   
4                         Anguilla             0.000000        0.000000e+00   

                Equacao_Linear  
0          y = 0.20x + 

In [18]:
# 1. Identar colunas de  2009 a 2023
year_cols = [str(year) for year in range(2009, 2024)]

# 2. Melt df_exportacao_filtrada para vetor
df_exportacao_long = exportacao_valor.melt(
    id_vars=['Id', 'País'],
    value_vars=year_cols,
    var_name='Ano',
    value_name='Valor_Exportado'
)

# 3. Converter ano como inteiro
df_exportacao_long['Ano'] = pd.to_numeric(df_exportacao_long['Ano'])

print("DataFrame 'df_exportacao_long' after melting:")
print(df_exportacao_long.head())

# 4. Criar listas vazias para guardar os resultados
country_names = []
angular_coefficients = []
linear_coefficients = []
linear_equations = []

# 5. Iterar cada país
for country in df_exportacao_long['País'].unique():
    # 6a. Filtrar por país
    country_df = df_exportacao_long[df_exportacao_long['País'] == country]

    # 6b. Verificar se há dados suficiente para regressão
    if len(country_df) < 2:
        country_names.append(country)
        angular_coefficients.append(None)
        linear_coefficients.append(None)
        linear_equations.append(f"{country}: Insufficient data for regression")
        continue

    # 6c. Preparar os dados
    X = country_df['Ano'].values.reshape(-1, 1)  # Independent variable (Ano)
    y = country_df['Valor_Exportado'].values    # Dependent variable (Valor_Exportado)

    # 6d. Ajuste
    model = LinearRegression()
    model.fit(X, y)

    # 6e. Coeficientes
    coef_angular = model.coef_[0]
    coef_linear = model.intercept_

    # 6f. Formatar equação
    equation = f"y = {coef_angular:.2f}x + {coef_linear:.2f}"

    # 6g. Guardar nas listas
    country_names.append(country)
    angular_coefficients.append(coef_angular)
    linear_coefficients.append(coef_linear)
    linear_equations.append(equation)

# Criar um novo DataFrame com os resultados
df_regression_results = pd.DataFrame({
    'País': country_names,
    'Coeficiente_Angular': angular_coefficients,
    'Coeficiente_Linear': linear_coefficients,
    'Equacao_Linear': linear_equations
})

print("\nFirst 5 rows of the regression results DataFrame:")
print(df_regression_results.head())

# Exportar como CSV
arquivo = 'exportacao_reg_valor.csv'
if not os.path.exists(arquivo):
    df_regression_results.to_csv(arquivo, index=False)
    print(f"DataFrame 'exportacao_reg_valor' exportado para '{arquivo}'.")
else:
    print(f"Arquivo '{arquivo}' já está gerado.")

DataFrame 'df_exportacao_long' after melting:
   Id                             País   Ano  Valor_Exportado
0   1                      Afeganistão  2009                0
1   2                    África do Sul  2009                0
2   3  Alemanha, República Democrática  2009           393482
3   4                           Angola  2009            84235
4   5                         Anguilla  2009                0

First 5 rows of the regression results DataFrame:
                              País  Coeficiente_Angular  Coeficiente_Linear  \
0                      Afeganistão             0.821429       -1.652933e+03   
1                    África do Sul            18.767857       -3.778173e+04   
2  Alemanha, República Democrática        -21372.653571        4.323049e+07   
3                           Angola         -7520.778571        1.519556e+07   
4                         Anguilla             0.000000        0.000000e+00   

                 Equacao_Linear  
0          y = 0.82x +

### Importação

In [19]:
# 1. Identar colunas de  2009 a 2023
year_cols = [str(year) for year in range(2009, 2024)]

# 2. Melt df_exportacao_filtrada para vetor
df_importacao_long = importacao_qtd.melt(
    id_vars=['Id', 'País'],
    value_vars=year_cols,
    var_name='Ano',
    value_name='Valor_Exportado'
)

# 3. Converter ano como inteiro
df_importacao_long['Ano'] = pd.to_numeric(df_importacao_long['Ano'])

print("DataFrame 'df_importacao_long' after melting:")
print(df_importacao_long.head())

# 4. Criar listas vazias para guardar os resultados
country_names = []
angular_coefficients = []
linear_coefficients = []
linear_equations = []

# 5. Iterar cada país
for country in df_importacao_long['País'].unique():
    # 6a. Filtrar por país
    country_df = df_importacao_long[df_importacao_long['País'] == country]

    # 6b. Verificar se há dados suficiente para regressão
    if len(country_df) < 2:
        country_names.append(country)
        angular_coefficients.append(None)
        linear_coefficients.append(None)
        linear_equations.append(f"{country}: Insufficient data for regression")
        continue

    # 6c. Preparar os dados
    X = country_df['Ano'].values.reshape(-1, 1)  # Independent variable (Ano)
    y = country_df['Valor_Exportado'].values    # Dependent variable (Valor_Exportado)

    # 6d. Ajuste
    model = LinearRegression()
    model.fit(X, y)

    # 6e. Coeficientes
    coef_angular = model.coef_[0]
    coef_linear = model.intercept_

    # 6f. Formatar equação
    equation = f"y = {coef_angular:.2f}x + {coef_linear:.2f}"

    # 6g. Guardar nas listas
    country_names.append(country)
    angular_coefficients.append(coef_angular)
    linear_coefficients.append(coef_linear)
    linear_equations.append(equation)

# Criar um novo DataFrame com os resultados
df_regression_results = pd.DataFrame({
    'País': country_names,
    'Coeficiente_Angular': angular_coefficients,
    'Coeficiente_Linear': linear_coefficients,
    'Equacao_Linear': linear_equations
})

print("\nFirst 5 rows of the regression results DataFrame:")
print(df_regression_results.head())

# Exportar como CSV
arquivo = 'importacao_reg_qtd.csv'
if not os.path.exists(arquivo):
    df_regression_results.to_csv(arquivo, index=False)
    print(f"DataFrame 'importacao_reg_qtd' exportado para '{arquivo}'.")
else:
    print(f"Arquivo '{arquivo}' já está gerado.")

DataFrame 'df_importacao_long' after melting:
   Id            País   Ano  Valor_Exportado
0   1   Africa do Sul  2009           493093
1   2        Alemanha  2009            56075
2   3         Argélia  2009                0
3   4  Arábia Saudita  2009                0
4   5       Argentina  2009         13902750

First 5 rows of the regression results DataFrame:
             País  Coeficiente_Angular  Coeficiente_Linear  \
0   Africa do Sul         11679.707143       -2.279200e+07   
1        Alemanha         -2275.371429        4.707824e+06   
2         Argélia             0.000000        0.000000e+00   
3  Arábia Saudita            49.042857       -9.866500e+04   
4       Argentina        881032.278571       -1.758504e+09   

                    Equacao_Linear  
0     y = 11679.71x + -22791995.60  
1       y = -2275.37x + 4707824.47  
2                 y = 0.00x + 0.00  
3           y = 49.04x + -98665.00  
4  y = 881032.28x + -1758503900.67  
Arquivo 'importacao_reg_qtd.csv' já es

In [20]:
# 1. Identar colunas de  2009 a 2023
year_cols = [str(year) for year in range(2009, 2024)]

# 2. Melt df_exportacao_filtrada para vetor
df_importacao_long = importacao_valor.melt(
    id_vars=['Id', 'País'],
    value_vars=year_cols,
    var_name='Ano',
    value_name='Valor_Exportado'
)

# 3. Converter ano como inteiro
df_importacao_long['Ano'] = pd.to_numeric(df_importacao_long['Ano'])

print("DataFrame 'df_importacao_long' after melting:")
print(df_importacao_long.head())

# 4. Criar listas vazias para guardar os resultados
country_names = []
angular_coefficients = []
linear_coefficients = []
linear_equations = []

# 5. Iterar cada país
for country in df_importacao_long['País'].unique():
    # 6a. Filtrar por país
    country_df = df_importacao_long[df_importacao_long['País'] == country]

    # 6b. Verificar se há dados suficiente para regressão
    if len(country_df) < 2:
        country_names.append(country)
        angular_coefficients.append(None)
        linear_coefficients.append(None)
        linear_equations.append(f"{country}: Insufficient data for regression")
        continue

    # 6c. Preparar os dados
    X = country_df['Ano'].values.reshape(-1, 1)  # Independent variable (Ano)
    y = country_df['Valor_Exportado'].values    # Dependent variable (Valor_Exportado)

    # 6d. Ajuste
    model = LinearRegression()
    model.fit(X, y)

    # 6e. Coeficientes
    coef_angular = model.coef_[0]
    coef_linear = model.intercept_

    # 6f. Formatar equação
    equation = f"y = {coef_angular:.2f}x + {coef_linear:.2f}"

    # 6g. Guardar nas listas
    country_names.append(country)
    angular_coefficients.append(coef_angular)
    linear_coefficients.append(coef_linear)
    linear_equations.append(equation)

# Criar um novo DataFrame com os resultados
df_regression_results = pd.DataFrame({
    'País': country_names,
    'Coeficiente_Angular': angular_coefficients,
    'Coeficiente_Linear': linear_coefficients,
    'Equacao_Linear': linear_equations
})

print("\nFirst 5 rows of the regression results DataFrame:")
print(df_regression_results.head())

# Exportar como CSV
arquivo = 'importacao_ref_valor.csv'
if not os.path.exists(arquivo):
    df_regression_results.to_csv(arquivo, index=False)
    print(f"DataFrame 'importacao_reg_valor' exportado para '{arquivo}'.")
else:
    print(f"Arquivo '{arquivo}' já está gerado.")

DataFrame 'df_importacao_long' after melting:
   Id            País   Ano  Valor_Exportado
0   1   Africa do Sul  2009          1931449
1   2        Alemanha  2009           291876
2   3         Argélia  2009                0
3   4  Arábia Saudita  2009                0
4   5       Argentina  2009         38521731

First 5 rows of the regression results DataFrame:
             País  Coeficiente_Angular  Coeficiente_Linear  \
0   Africa do Sul        -2.397772e+04        5.094178e+07   
1        Alemanha        -8.834604e+03        1.832767e+07   
2         Argélia         5.307881e-17        2.866667e+00   
3  Arábia Saudita         1.836786e+02       -3.694817e+05   
4       Argentina         2.475842e+06       -4.932759e+09   

                     Equacao_Linear  
0      y = -23977.72x + 50941781.80  
1       y = -8834.60x + 18327671.53  
2                  y = 0.00x + 2.87  
3          y = 183.68x + -369481.73  
4  y = 2475842.43x + -4932758886.00  
Arquivo 'importacao_ref_valor.cs